# Raccolta features per TEAMS

In [1]:
import pymongo
from pymongo import MongoClient

from pandas import DataFrame, read_csv
import json
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

client = MongoClient('mongodb://127.0.0.1:27017')
db = client.sample

cu = db.users.find()
cs = db.shots.find()
cf = db.followers.find()
cl = db.likes.find()
ct = db.teams.find()
cutfs = db.user_team_first_shot.find()
csk = db.skills.find()

In [2]:
usernames_no_repeated = []
for user in cutfs:
    if user['user'] not in usernames_no_repeated:
        usernames_no_repeated.append(user['user'])

In [ ]:
usernames_no_repeated[0]

In [ ]:
db.user_team_first_shot.find({'user':'citrusbyte'})[0]

In [ ]:
len(usernames_no_repeated)

### Media degli shots di uno User in un Team

In [ ]:
shot_number_for_user_in_team = []

i=0

for u in usernames_no_repeated:
    active_user = db.shots.find({'username': u})
    if active_user.count() > 0:
        temp_cursor = db.user_team_first_shot.find({'user': active_user[0]['username']})
        for t in temp_cursor:
            elem = {'team': t['team'], 'user': t['user'], 'n_shots': len(active_user[0]['shots'])}
            shot_number_for_user_in_team.append(elem)
    
    i=i+1
    print i

In [ ]:
len(shot_number_for_user_in_team)

scrivere su file 

In [ ]:
with open("/home/simo/UnInfo/Tesi Magistrale/Dataset/shot_number_for_user_in_team.txt", mode='a') as f:
    for s in shot_number_for_user_in_team:
        f.write(str(s['team'])+' - '+str(s['user'])+' - '+str(s['n_shots'])+'\n')

leggere da file

In [3]:
def find_nth(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

In [4]:
shot_number_for_user_in_team = []

with open("/home/simo/UnInfo/Tesi Magistrale/Dataset/features/shot_number_for_user_in_team.txt", "r") as ins:
    for line in ins:
        pr_occ = find_nth(line, '-', 1)
        
        n = line.count('-')
        sec_occ = find_nth(line, '-', n)
        
        team = line[0:pr_occ-1]
        user = line[pr_occ+2:sec_occ-1]
        n_shots = line[sec_occ+2:len(line)-1]
        
        elem = {'team': team, 'user': user, 'n_shots': n_shots}
        shot_number_for_user_in_team.append(elem)

numero medio di shot per utente in un team

In [5]:
def mean_value_of_shots(team):
    count = 0
    users_in_team = 0
    for t in shot_number_for_user_in_team:
        if t['team'] == team:
            users_in_team = users_in_team + 1;
            count = count + int(t['n_shots'])
    
    print 'MEAN VALUE OF SHOTS IN TEAM '+team+' :'
    print float(count)/float(users_in_team)    

In [6]:
def value_of_shots(team):
    count = 0
    for t in shot_number_for_user_in_team:
        if t['team'] == team:
            count = count + int(t['n_shots'])
    
    return count

In [11]:
count = 0
for t in ct:
    count = count + value_of_shots(t['username'])
    
print 'MEAN VALUE OF SHOTS PER TEAMS: ' #il numero medio di shots tra tutti i team
print float(count)/float(ct.count())

MEAN VALUE OF SHOTS PER TEAMS: 
0.0


/home/simo/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


team con il maggior numero di utenti

In [3]:
max_users = 0
team_max = {}
for t in ct:
    if int(t['members_count']) > max_users:
        max_users = int(t['members_count'])
        team_max = t
        
print 'numero massimo', max_users
team_max

numero massimo 85


{u'_id': ObjectId('5bb259b1e2850521dc2f1a49'),
 u'avatar_url': u'https://cdn.dribbble.com/users/235795/avatars/normal/81e287f1f8fb7aedd2ca1d3ad7078c73.jpeg?1509466931',
 u'bio': u'Shopify is everything you need to sell online, in-store, and anywhere in between. Making commerce better for everyone. ',
 u'buckets_count': 0,
 u'buckets_url': u'https://api.dribbble.com/v1/users/235795/buckets',
 u'can_upload_shot': True,
 u'comments_received_count': 60,
 u'created_at': 1352740507,
 u'followers_count': 7593,
 u'followers_url': u'https://api.dribbble.com/v1/users/235795/followers',
 u'following_url': u'https://api.dribbble.com/v1/users/235795/following',
 u'followings_count': 132,
 u'html_url': u'https://dribbble.com/Shopify',
 u'id': 235795,
 u'likes_count': 108,
 u'likes_received_count': 1329,
 u'likes_url': u'https://api.dribbble.com/v1/users/235795/likes',
 u'links': {u'twitter': u'https://twitter.com/shopifyUX',
  u'web': u'http://ux.shopify.com'},
 u'location': u'Canada',
 u'members_co

In [7]:
max = 0
max_team = {}
for t in ct:
    if value_of_shots(t['username']) > max:
        max = value_of_shots(t['username'])
        max_team = t
    
print 'TEAM WITH MORE SHOTS: '
print 'NUMBER OF SHOTS: '+str(max)
print max_team['username']

TEAM WITH MORE SHOTS: 
NUMBER OF SHOTS: 5983
Slaptastick


In [16]:
db.teams.find()[0]

{u'_id': ObjectId('5bb259ace2850521dc2f1856'),
 u'avatar_url': u'https://cdn.dribbble.com/users/673130/avatars/normal/4fb0be803bf3f04fdf007117e4622bfc.png?1507932552',
 u'bio': u'We are a global team of strategy, design and development experts.',
 u'buckets_count': 0,
 u'buckets_url': u'https://api.dribbble.com/v1/users/673130/buckets',
 u'can_upload_shot': True,
 u'comments_received_count': 0,
 u'created_at': 1413503476,
 u'followers_count': 1042,
 u'followers_url': u'https://api.dribbble.com/v1/users/673130/followers',
 u'following_url': u'https://api.dribbble.com/v1/users/673130/following',
 u'followings_count': 1943,
 u'html_url': u'https://dribbble.com/citrusbyte',
 u'id': 673130,
 u'likes_count': 471,
 u'likes_received_count': 40,
 u'likes_url': u'https://api.dribbble.com/v1/users/673130/likes',
 u'links': {u'twitter': u'https://twitter.com/citrusbyte',
  u'web': u'https://citrusbyte.com'},
 u'location': u'Los Angeles',
 u'members_count': 18,
 u'members_url': u'https://api.dribbb

In [8]:
mean_value_of_shots('Slaptastick')

MEAN VALUE OF SHOTS IN TEAM Slaptastick :
299.15


In [9]:
mean_value_of_shots('dropbox')

MEAN VALUE OF SHOTS IN TEAM dropbox :
90.8571428571


shots di dropbox

In [8]:
value_of_shots('dropbox')

4452